# 🛠️ Preprocesamiento de Datos
## Segmentación de Clientes con Tarjetas de Crédito

---

### 📌 Información del Notebook

- **Objetivo**: Limpiar y preparar los datos para el clustering
- **Acciones**: Manejo de nulos, normalización, detección de outliers
- **Autor**: [Tu Nombre]
- **Fecha**: Enero 2026

---

### 🎯 Contenido

1. Carga de datos
2. Manejo de valores nulos
3. Eliminación de duplicados
4. Análisis y tratamiento de outliers
5. Normalización de datos
6. Reducción de dimensionalidad (PCA)
7. Guardado de datos procesados


In [ ]:
# Importamos las librerías para manipulación de datos, preprocesamiento y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Librerías importadas correctamente")

---
## 📁 1. Carga del Dataset Original

### Cargar datos desde el archivo CSV


In [ ]:
# Cargamos el dataset original que analizamos en el notebook anterior
# El archivo debe estar en la carpeta datos/
df_original = pd.read_csv('../datos/CC_GENERAL.csv')

print("="*70)
print("📊 DATASET CARGADO")
print("="*70)
print(f"Filas: {df_original.shape[0]:,}")
print(f"Columnas: {df_original.shape[1]}")
print("="*70)

### Crear copia para trabajar sin modificar el original


In [ ]:
# Creamos una copia del dataframe para no modificar los datos originales
# Esto nos permite volver atrás si es necesario
df = df_original.copy()

print(f"✅ Copia de trabajo creada: {df.shape}")

---
## 🗑️ 2. Eliminación de Columnas Innecesarias

### Eliminar la columna CUST_ID


In [ ]:
# Eliminamos la columna CUST_ID porque:
# 1. Es un identificador único que no aporta información para clustering
# 2. No tiene valor predictivo
# 3. Puede sesgar los algoritmos de ML
if 'CUST_ID' in df.columns:
    df = df.drop('CUST_ID', axis=1)
    print("✅ Columna CUST_ID eliminada")
    print(f"   Nuevas dimensiones: {df.shape}")
else:
    print("⚠️ CUST_ID no encontrada en el dataset")

---
## 🔍 3. Análisis de Valores Nulos

### Identificar columnas con valores nulos


In [ ]:
# Verificamos qué columnas tienen valores nulos y en qué cantidad
# Esto nos ayuda a decidir la estrategia de imputación
nulos_total = df.isnull().sum()
nulos_df = pd.DataFrame({
    'Columna': nulos_total.index,
    'Nulos': nulos_total.values,
    '% Nulos': (nulos_total.values / len(df) * 100).round(2)
})

nulos_df = nulos_df[nulos_df['Nulos'] > 0].sort_values('Nulos', ascending=False)

print("🔎 ANÁLISIS DE VALORES NULOS")
print("="*70)
if len(nulos_df) > 0:
    print("\n⚠️ Columnas con valores nulos:")
    print(nulos_df.to_string(index=False))
    print(f"\nTotal de valores nulos: {df.isnull().sum().sum():,}")
else:
    print("✅ No hay valores nulos")
print("="*70)

### Imputar valores nulos con la mediana


In [ ]:
# Imputamos los valores nulos usando la mediana porque:
# 1. La mediana es robusta a outliers (nuestro dataset tiene muchos)
# 2. Es mejor que la media para distribuciones asimétricas
# 3. Preserva la distribución original de los datos
if df.isnull().sum().sum() > 0:
    imputer = SimpleImputer(strategy='median')
    columnas = df.columns
    df_imputado = pd.DataFrame(
        imputer.fit_transform(df),
        columns=columnas
    )
    df = df_imputado
    print("✅ Valores nulos imputados con la mediana")
    print(f"   Verificación: {df.isnull().sum().sum()} nulos restantes")
else:
    print("✅ No hay valores nulos que imputar")

---
## 🔄 4. Eliminación de Duplicados

### Identificar y eliminar registros duplicados


In [ ]:
# Verificamos y eliminamos registros duplicados
# Los duplicados pueden sesgar el clustering al dar más peso a ciertos patrones
duplicados_antes = df.duplicated().sum()

if duplicados_antes > 0:
    df = df.drop_duplicates()
    duplicados_despues = df.duplicated().sum()
    
    print("🔄 ELIMINACIÓN DE DUPLICADOS")
    print("="*70)
    print(f"Duplicados encontrados: {duplicados_antes}")
    print(f"Registros eliminados: {duplicados_antes - duplicados_despues}")
    print(f"Duplicados restantes: {duplicados_despues}")
    print(f"Nuevas dimensiones: {df.shape}")
    print("="*70)
else:
    print("✅ No hay registros duplicados")

---
## 📊 5. Análisis de Outliers

### Detectar outliers usando el método IQR (Rango Intercuartílico)


In [ ]:
# Función para detectar outliers usando el método IQR
# IQR = Q3 - Q1 (rango entre percentil 75 y 25)
# Outliers: valores < Q1 - 1.5*IQR o > Q3 + 1.5*IQR
def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    
    return len(outliers), limite_inferior, limite_superior

# Aplicamos la detección a todas las columnas numéricas
print("🔍 DETECCIÓN DE OUTLIERS (Método IQR)")
print("="*80)

outliers_resumen = []
for col in df.columns:
    n_outliers, lim_inf, lim_sup = detectar_outliers_iqr(df, col)
    porcentaje = (n_outliers / len(df)) * 100
    
    outliers_resumen.append({
        'Variable': col,
        'N_Outliers': n_outliers,
        '%_Outliers': round(porcentaje, 2)
    })

df_outliers = pd.DataFrame(outliers_resumen)
df_outliers = df_outliers.sort_values('N_Outliers', ascending=False)

print(df_outliers.to_string(index=False))
print("="*80)

### Visualizar outliers en variables clave


In [ ]:
# Creamos boxplots para visualizar los outliers en las variables más importantes
# Esto nos ayuda a decidir si los mantenemos o eliminamos
variables_clave = ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS', 'CASH_ADVANCE']

fig, axes = plt.subplots(1, 5, figsize=(20, 5))

for idx, var in enumerate(variables_clave):
    # Creamos un boxplot para cada variable
    axes[idx].boxplot(df[var].dropna(), vert=True)
    axes[idx].set_title(f'{var}', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Valor')
    axes[idx].grid(True, alpha=0.3)
    
    # Calculamos cantidad de outliers
    n_out, _, _ = detectar_outliers_iqr(df, var)
    axes[idx].text(0.5, 0.98, f'Outliers: {n_out}', 
                   transform=axes[idx].transAxes,
                   fontsize=9, ha='center', va='top',
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.suptitle('Outliers en Variables Clave', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Decisión sobre outliers


In [ ]:
# DECISIÓN: NO eliminamos outliers porque:
# 1. En datos de tarjetas de crédito, los outliers pueden representar segmentos reales
#    (clientes VIP, usuarios frecuentes, etc.)
# 2. El clustering puede identificar estos grupos como clusters separados
# 3. La normalización reducirá su impacto
# 4. Eliminarlos podría perder información valiosa de negocio

print("⚠️ DECISIÓN SOBRE OUTLIERS:")
print("="*70)
print("Los outliers NO serán eliminados porque:")
print("  1. Pueden representar segmentos de clientes reales")
print("  2. Son valiosos para identificar clusters especiales (VIP, etc.)")
print("  3. La normalización mitigará su impacto")
print("="*70)

---
## 📏 6. Normalización de Datos

### Aplicar StandardScaler para normalizar todas las variables


In [ ]:
# Normalizamos los datos usando StandardScaler
# Esto transforma cada variable a media=0 y desviación estándar=1
# Es CRUCIAL para clustering porque:
# 1. Las variables tienen escalas muy diferentes (TENURE: 0-12 vs BALANCE: 0-20,000)
# 2. KMeans es sensible a las escalas
# 3. Evita que variables con valores grandes dominen el clustering
scaler = StandardScaler()
datos_normalizados = scaler.fit_transform(df)

df_normalizado = pd.DataFrame(
    datos_normalizados,
    columns=df.columns
)

print("✅ DATOS NORMALIZADOS CON STANDARDSCALER")
print("="*70)
print(f"Dimensiones: {df_normalizado.shape}")
print(f"\nVerificación de normalización (debe ser ≈0 y ≈1):")
print(f"  Media de todas las variables: {df_normalizado.mean().mean():.6f}")
print(f"  Desviación estándar promedio: {df_normalizado.std().mean():.6f}")
print("="*70)

### Comparar datos antes y después de normalización


In [ ]:
# Comparamos una variable antes y después de normalizar
# Esto nos permite visualizar el efecto de la normalización
variable_ejemplo = 'BALANCE'

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Antes de normalizar
axes[0].hist(df[variable_ejemplo], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0].set_title(f'{variable_ejemplo} - ANTES de Normalizar', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Valor Original')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True, alpha=0.3)

# Después de normalizar
axes[1].hist(df_normalizado[variable_ejemplo], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[1].set_title(f'{variable_ejemplo} - DESPUÉS de Normalizar', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Valor Normalizado (z-score)')
axes[1].set_ylabel('Frecuencia')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Efecto de la Normalización', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"✅ Variable {variable_ejemplo}:")
print(f"   Antes  → Media: {df[variable_ejemplo].mean():.2f}, Std: {df[variable_ejemplo].std():.2f}")
print(f"   Después → Media: {df_normalizado[variable_ejemplo].mean():.6f}, Std: {df_normalizado[variable_ejemplo].std():.6f}")

---
## 🎯 7. Reducción de Dimensionalidad (PCA)

### Aplicar PCA para visualización y análisis


In [ ]:
# Aplicamos PCA (Principal Component Analysis) para:
# 1. Reducir dimensionalidad (17 variables → 2-3 componentes principales)
# 2. Facilitar la visualización de clusters
# 3. Capturar la mayor varianza posible con menos variables
# 4. Reducir ruido y redundancia

# Primero probamos con todos los componentes para ver varianza explicada
pca_completo = PCA()
pca_completo.fit(df_normalizado)

varianza_explicada = pca_completo.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

print("📊 ANÁLISIS PCA - VARIANZA EXPLICADA")
print("="*70)
for i in range(min(10, len(varianza_explicada))):
    print(f"PC{i+1}: {varianza_explicada[i]*100:6.2f}% | Acumulada: {varianza_acumulada[i]*100:6.2f}%")
print("="*70)

### Visualizar varianza explicada (Scree Plot)


In [ ]:
# Creamos un Scree Plot para decidir cuántos componentes usar
# Este gráfico muestra cuánta varianza explica cada componente
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Gráfico 1: Varianza por componente
axes[0].plot(range(1, len(varianza_explicada)+1), 
             varianza_explicada * 100, 
             marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0].set_title('Scree Plot - Varianza por Componente', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Componente Principal')
axes[0].set_ylabel('Varianza Explicada (%)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=10, color='red', linestyle='--', label='10% umbral')
axes[0].legend()

# Gráfico 2: Varianza acumulada
axes[1].plot(range(1, len(varianza_acumulada)+1), 
             varianza_acumulada * 100, 
             marker='s', linewidth=2, markersize=8, color='green')
axes[1].set_title('Varianza Explicada Acumulada', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Número de Componentes')
axes[1].set_ylabel('Varianza Acumulada (%)')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=80, color='red', linestyle='--', label='80% objetivo')
axes[1].legend()

plt.tight_layout()
plt.show()

# Mostramos cuántos componentes necesitamos para 80% y 90% de varianza
for umbral in [0.80, 0.90]:
    n_comp = np.argmax(varianza_acumulada >= umbral) + 1
    print(f"✅ Para {umbral*100:.0f}% varianza → {n_comp} componentes")

### Aplicar PCA con componentes óptimos


In [ ]:
# Aplicamos PCA con el número óptimo de componentes
# Usaremos los componentes que explican al menos 80% de la varianza
n_componentes_optimo = np.argmax(varianza_acumulada >= 0.80) + 1

pca = PCA(n_components=n_componentes_optimo)
datos_pca = pca.fit_transform(df_normalizado)

df_pca = pd.DataFrame(
    datos_pca,
    columns=[f'PC{i+1}' for i in range(n_componentes_optimo)]
)

print(f"✅ PCA APLICADO")
print("="*70)
print(f"Componentes principales: {n_componentes_optimo}")
print(f"Varianza explicada total: {pca.explained_variance_ratio_.sum()*100:.2f}%")
print(f"Dimensiones reducidas: {df.shape[1]} → {df_pca.shape[1]} variables")
print(f"Forma de datos PCA: {df_pca.shape}")
print("="*70)

### Visualizar contribución de variables a componentes principales


In [ ]:
# Analizamos qué variables originales contribuyen más a cada componente principal
# Esto nos ayuda a interpretar el significado de cada PC
componentes_df = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(n_componentes_optimo)],
    index=df.columns
)

# Mostramos las 5 variables más importantes para PC1 y PC2
print("\n📊 CONTRIBUCIÓN DE VARIABLES A COMPONENTES PRINCIPALES\n")
print("="*70)

for pc in ['PC1', 'PC2']:
    print(f"\n{pc} - Top 5 variables más influyentes:")
    top_vars = componentes_df[pc].abs().sort_values(ascending=False).head(5)
    for var, peso in top_vars.items():
        print(f"  {var:35s}: {componentes_df.loc[var, pc]:7.3f}")

print("="*70)

In [ ]:
# Heatmap de contribución de variables a componentes
plt.figure(figsize=(10, 12))
sns.heatmap(
    componentes_df[['PC1', 'PC2', 'PC3']],
    annot=True,
    fmt='.2f',
    cmap='RdBu_r',
    center=0,
    linewidths=0.5,
    cbar_kws={'label': 'Peso del componente'}
)
plt.title('Contribución de Variables a los Primeros 3 Componentes Principales', 
          fontsize=13, fontweight='bold', pad=15)
plt.xlabel('Componentes Principales')
plt.ylabel('Variables Originales')
plt.tight_layout()
plt.show()

---
## 💾 8. Guardar Datos Procesados

### Exportar datos limpios y normalizados


In [ ]:
# Guardamos los datos procesados en diferentes formatos para el siguiente notebook
# 1. Datos normalizados (para clustering)
# 2. Datos PCA (para visualización)
# 3. Objeto scaler (para nuevos datos)

# Guardar datos normalizados
df_normalizado.to_csv('../datos/datos_normalizados.csv', index=False)
print("✅ Datos normalizados guardados: datos/datos_normalizados.csv")

# Guardar datos con PCA
df_pca.to_csv('../datos/datos_pca.csv', index=False)
print("✅ Datos PCA guardados: datos/datos_pca.csv")

# Guardar datos procesados (sin normalizar) para análisis posterior
df.to_csv('../datos/datos_procesados.csv', index=False)
print("✅ Datos procesados guardados: datos/datos_procesados.csv")

---
## 📋 Resumen del Preprocesamiento

### ✅ Acciones Realizadas:

1. **Eliminación de columnas**: CUST_ID removido
2. **Valores nulos**: Imputados con la mediana
3. **Duplicados**: Eliminados (si existían)
4. **Outliers**: Mantenidos (importantes para clustering)
5. **Normalización**: StandardScaler aplicado (media=0, std=1)
6. **PCA**: Reducción a componentes que explican 80%+ de varianza

### 📊 Resultado Final:

- **Registros limpios**: ~8,950 clientes
- **Variables normalizadas**: 17 features
- **Componentes PCA**: Variables reducidas
- **Datos listos para**: Clustering K-Means

### 📁 Archivos Generados:

- `datos_procesados.csv` → Datos limpios sin normalizar
- `datos_normalizados.csv` → Datos listos para clustering
- `datos_pca.csv` → Datos con PCA para visualización

---

**Próximo paso**: Notebook 3 - Clustering con K-Means
